In [7]:
import pandas as pd
df_lat_lon = pd.read_csv(r'C:\Users\ABRASEL NACIONAL\Documents\GitHub\ESTABCHECK\auxiliares\MUNICIPIOS_LAT_LON.csv', sep=";", dtype="str")
df_municipios = pd.read_csv(r'C:\Users\ABRASEL NACIONAL\Documents\GitHub\ESTABCHECK\auxiliares\MUNICIPIOS.CSV', sep=";",dtype="str")
dados = pd.merge(df_lat_lon, df_municipios, how='inner', on="MUNICIPIO")
dados.rename(columns={"COD_UF_y": "COD_UF", "UF_y": "UF"}, inplace=True)
dados = dados[["MUNICIPIO", "CIDADE", "COD_UF", "UF", "ESTADO", "REGIAO", "BAIRRO", "LATITUDE", "LONGITUDE"]]
dados

,MUNICIPIO,CIDADE,COD_UF,UF,ESTADO,REGIAO,BAIRRO,LATITUDE,LONGITUDE
0,1100015,ALTA FLORESTA D'OESTE,11,RO,RONDÔNIA,NORTE,POSTO FISCAL ROLIM DE MOURA,-13.08271,-62.27726
1,1100015,ALTA FLORESTA D'OESTE,11,RO,RONDÔNIA,NORTE,ALTA FLORESTA D'OESTE,-11.96806,-61.95417
2,1100015,ALTA FLORESTA D'OESTE,11,RO,RONDÔNIA,NORTE,CLIZA,-12.61667,-62.06667
3,1100015,ALTA FLORESTA D'OESTE,11,RO,RONDÔNIA,NORTE,URUROSES,-12.86667,-62.2
4,1100015,ALTA FLORESTA D'OESTE,11,RO,RONDÔNIA,NORTE,CONSUELO,-12.65,-62.13333
...,...,...,...,...,...,...,...,...,...
541908,5300108,BRASÍLIA,53,DF,DISTRITO FEDERAL,CENTRO-OESTE,CONDOMINIO RESIDENCIAL SOBRADINHO III,-15.64249,-47.83287
541909,5300108,BRASÍLIA,53,DF,DISTRITO FEDERAL,CENTRO-OESTE,SETOR CULTURAL SUL,-15.7963,-47.88241
541910,5300108,BRASÍLIA,53,DF,DISTRITO FEDERAL,CENTRO-OESTE,SETOR CULTURAL NORTE,-15.79151,-47.88391
541911,5300108,BRASÍLIA,53,DF,DISTRITO FEDERAL,CENTRO-OESTE,SETOR HOTELEIRO SUL,-15.79436,-47.88979


In [ ]:
import pandas as pd
import sqlalchemy
dados = pd.read_parquet('br_base\Restaurantes e similares.gzip')
dados.columns

In [ ]:
import duckdb

# Criando um objeto duckdb
duck = duckdb.connect()

# consulta
query = """
SELECT DATA_SITUACAO_CADASTRAL, 
COUNT(*) as TOTAL
FROM dados 
WHERE CNAE_DESCRICAO = 'RESTAURANTES E SIMILARES'
GROUP BY DATA_SITUACAO_CADASTRAL;
"""

# Exemplo de consulta SQL usando duckdb
resultado = duck.query(query).df()

resultado

In [ ]:
ultima_alteracao = duck.query("""
SELECT DATA_SITUACAO_CADASTRAL 
FROM dados
ORDER BY DATA_SITUACAO_CADASTRAL DESC
LIMIT 1;
""").fetchone()[0]

ultima_alteracao